# Cipher Information

In [1]:
classic_16_solve_times = []
classic_32_solve_times = []
classic_64_solve_times = []
#classic_128_solve_times = []

quantum_16_solve_times = []
quantum_32_solve_times = []
quantum_64_solve_times = []
#quantum_128_solve_times = []

In [2]:

# GIFT S-box
GIFT_SBOX = [1, 10, 4, 12, 6, 15, 3, 9, 2, 13, 11, 7, 5, 0, 8, 14]
# GIFT-16/32/64/128 Permutations, MSB, Source to Target representation
GIFT16_BITPERM = (12, 1, 6, 11, 8, 13, 2, 7, 4, 9, 14, 3, 0, 5, 10, 15)
GIFT32_BITPERM = (12, 1, 6, 11, 28, 17, 22, 27, 8, 13, 2, 7, 24, 29, 18, 23, 4, 9, 14, 3, 20, 25, 30, 19, 0, 5, 10, 15, 16, 21, 26, 31)
GIFT64_BITPERM = (12, 1, 6, 11, 28, 17, 22, 27, 44, 33, 38, 43, 60, 49, 54, 59, 8, 13, 2, 7, 24, 29, 18, 23, 40, 45, 34, 39, 56, 61, 50, 55, 4, 9, 14, 3, 20, 25, 30, 19, 36, 41, 46, 35, 52, 57, 62, 51, 0, 5, 10, 15, 16, 21, 26, 31, 32, 37, 42, 47, 48, 53, 58, 63)
#GIFT128_BITPERM = (12, 1, 6, 11, 28, 17, 22, 27, 44, 33, 38, 43, 60, 49, 54, 59, 76, 65, 70, 75, 92, 81, 86, 91, 108, 97, 102, 107, 124, 113, 118, 123, 8, 13, 2, 7, 24, 29, 18, 23, 40, 45, 34, 39, 56, 61, 50, 55, 72, 77, 66, 71, 88, 93, 82, 87, 104, 109, 98, 103, 120, 125, 114, 119, 4, 9, 14, 3, 20, 25, 30, 19, 36, 41, 46, 35, 52, 57, 62, 51, 68, 73, 78, 67, 84, 89, 94, 83, 100, 105, 110, 99, 116, 121, 126, 115, 0, 5, 10, 15, 16, 21, 26, 31, 32, 37, 42, 47, 48, 53, 58, 63, 64, 69, 74, 79, 80, 85, 90, 95, 96, 101, 106, 111, 112, 117, 122, 127)


# S-box Constraints for GIFT S-box

In [3]:

sbox_constraints = [
' a[0] + a[1] + a[2] + a[3] - b[0] - b[1] - b[2] - b[3] >= 0',
' - 3*a[0] - 3*a[1] - 5*a[2] - 4*a[3] + 2*b[0] + 3*b[1] + b[2] + b[3] >= -8',
' - 3*a[0] - 2*a[1] + 3*a[2] - a[3] - b[0] - 2*b[1] - 4*b[2] + 3*b[3] >= -7',
' - a[0] - a[1] - a[2] + 2*b[0] + 3*b[1] + b[2] + b[3] >= 0',
' 3*a[3] - b[0] - 2*b[1] - b[2] - b[3] >= -2',
' - a[1] - 2*a[3] - b[0] + b[1] + 2*b[2] - 2*b[3] >= -4',
' a[0] - a[3] + b[0] - b[1] - 2*b[2] - b[3] >= -3',
' - 3*a[0] - a[1] - 5*a[2] - 6*a[3] + 2*b[0] + b[1] + 5*b[2] + 3*b[3] >= -8',
' a[1] + 3*a[2] + a[3] - 2*b[0] - 2*b[1] - b[2] - 2*b[3] >= -2',
' a[1] + 3*a[3] - 2*b[0] - 2*b[1] - b[2] - b[3] >= -2',
' - a[0] - a[1] - a[3] + 3*b[0] + 2*b[1] + 2*b[2] + b[3] >= 0',
' - a[1] - a[3] - b[1] + b[2] + b[3] >= -2',
' 2*a[0] + a[1] + a[3] - b[0] - 2*b[1] - b[2] - 2*b[3] >= -2',
' - 2*a[1] - 2*a[2] - a[3] + b[0] + 2*b[1] + b[2] + b[3] >= -2',
' - a[0] - 2*a[3] - b[0] + b[1] - 2*b[2] + 2*b[3] >= -4']

# Classical Experiments

In [4]:
from ortools.linear_solver import pywraplp
import re
import time

class GenMILPModel:
    def __init__(self, solver_name: str = "GUROBI", num_threads: int = 1):
        self.solver = pywraplp.Solver.CreateSolver(solver_name)
        if self.solver is None:
            raise RuntimeError(f"Solver '{solver_name}' is not available.")
        self.rounds = []  # Each round: {'in': [...], 'out': [...]}
        self.constraints = []
        self.solve_times = []
        
        if num_threads is not None:
            try:
                self.solver.SetNumThreads(num_threads)
                print(f"[i] Solver threads set to {num_threads}")
            except Exception as e:
                print(f"[!] Solver does not support thread control: {e}")
                
    def add_round(self, sbox_size: int, n_sboxes: int, prefix: str = "x"):
        round_idx = len(self.rounds)
        n_bits = sbox_size * n_sboxes
        S_in = [self.solver.BoolVar(f"{prefix}_{round_idx}_in_{i}") for i in range(n_bits)]
        S_out = [self.solver.BoolVar(f"{prefix}_{round_idx}_out_{i}") for i in range(n_bits)]
        self.rounds.append({'in': S_in, 'out': S_out})
        return S_in, S_out

    def connect_rounds_with_permutation(self, prev_out, next_in, perm):
        assert len(prev_out) == len(next_in) == len(perm)
        for i in range(len(perm)):
            self.solver.Add(next_in[perm[i]] == prev_out[i])

    def add_constraint(self, coeffs: list[int], vars: list, rhs: int, sense: str = '=='):
        expr = sum(c * v for c, v in zip(coeffs, vars))
        if sense == '==':
            ct = self.solver.Add(expr == rhs)
        elif sense == '<=':
            ct = self.solver.Add(expr <= rhs)
        elif sense == '>=':
            ct = self.solver.Add(expr >= rhs)
        else:
            raise ValueError(f"Unsupported constraint type: {sense}")
        self.constraints.append(ct)

    def add_constraint_from_string(self, expr: str, a_vars, b_vars):
        if '>=' in expr:
            lhs, rhs = expr.split(">=")
            sense = '>='
        elif '<=' in expr:
            lhs, rhs = expr.split("<=")
            sense = '<='
        elif '==' in expr:
            lhs, rhs = expr.split("==")
            sense = '=='
        else:
            raise ValueError("Constraint must contain >=, <=, or ==")

        rhs = int(rhs.strip())
        pattern = re.compile(r'([+-]?\s*\d*\s*\*\s*[ab]\[\d+\]|[+-]?\s*[ab]\[\d+\])')
        terms = pattern.findall(lhs)

        coeffs = []
        vars_ = []

        for term in terms:
            term = term.replace(" ", "")
            if '*' in term:
                coef_str, var_str = term.split('*')
                coef = int(coef_str) if coef_str not in ('', '+') else 1
            else:
                coef = -1 if term.startswith('-') else 1
                var_str = term.lstrip('+-')

            var_type = var_str[0]
            index = int(var_str[2:-1])

            if var_type == 'a':
                v = a_vars[index]
            elif var_type == 'b':
                v = b_vars[index]
            else:
                raise ValueError(f"Unknown variable type: {var_type}")

            coeffs.append(coef)
            vars_.append(v)

        self.add_constraint(coeffs, vars_, rhs, sense)

    def add_sbox_layer(self, input_vars, output_vars, sbox_constraints, sbox_size, n_boxes):
        assert len(input_vars) == len(output_vars) == sbox_size * n_boxes

        for i in range(n_boxes):
            a_block = input_vars[i * sbox_size:(i + 1) * sbox_size]
            b_block = output_vars[i * sbox_size:(i + 1) * sbox_size]
            for expr in sbox_constraints:
                self.add_constraint_from_string(expr, a_block, b_block)

    def set_objective(self, coeffs: list[int], vars: list, sense: str = 'min'):
        expr = sum(c * v for c, v in zip(coeffs, vars))
        if sense == 'min':
            self.solver.Minimize(expr)
        elif sense == 'max':
            self.solver.Maximize(expr)
        else:
            raise ValueError(f"Unsupported objective direction: {sense}")

    def solve_for_integral_distinguisher(self, target_vars, blocksize=64, verbose=True):
        solver = self.solver
        counter = 0
        set_zero = []
        found = False
        integral_bits = []
        self.solve_times.clear() 
        while counter < blocksize:
            start = time.time()
            status = solver.Solve()
            elapsed = time.time() - start
            self.solve_times.append(elapsed) 
            if status == pywraplp.Solver.OPTIMAL:
                obj_val = solver.Objective().Value()
                if verbose:
                    print(f"[+] Solve #{counter} -> Objective: {obj_val}")
                if obj_val > 1:
                    found = True
                    integral_bits = [i for i, v in enumerate(target_vars) if v.solution_value() == 1]
                    break
                else:
                    for v in target_vars:
                        if v.solution_value() == 1:
                            if verbose:
                                print(f"  └─ Forcing {v.name()} = 0")
                            v.SetUb(0)
                            set_zero.append(v.name())
                            counter += 1
                            break
            elif status == pywraplp.Solver.INFEASIBLE:
                if verbose:
                    print("[X] Model is infeasible — distinguisher confirmed.")
                found = True
                break
            else:
                print("[!] Unknown error or model not solved.")
                break

        if found:
            print("\n[V] Integral Distinguisher FOUND!")
            if integral_bits:
                print("Indices with value 1:", integral_bits)
        else:
            print("\n[X] No integral distinguisher exists.")

        if verbose:
            print("Zero-forced bits:")
            if set_zero:
                for name in set_zero:
                    print(" ", name)
            elif counter == 0:
                print(" (none — distinguisher confirmed without forcing any bit)")
            else:
                print(" (none — model was immediately infeasible)")

    def solve(self, time_limit_sec: int = None):
        if time_limit_sec:
            self.solver.SetTimeLimit(time_limit_sec * 1000)
        return self.solver.Solve()

    def get_solution(self):
        return [
            {key: [int(v.solution_value()) for v in round_data[key]] for key in round_data}
            for round_data in self.rounds
        ]

    def print_solution(self):
        if self.solver.Solve() == pywraplp.Solver.OPTIMAL:
            print("Objective value:", self.solver.Objective().Value())
            for r, round_data in enumerate(self.rounds):
                for key in ['in', 'out']:
                    bits = [int(v.solution_value()) for v in round_data[key]]
                    print(f"Round {r} {key}: {bits}")
        else:
            print("No optimal solution found.")


# Application to GIFT-16

In [11]:
for idx in range(10):
    model = GenMILPModel("SCIP")

    SBOX_SIZE = 4
    N_SBOXES = 4
    BLOCKSIZE = SBOX_SIZE * N_SBOXES
    BITPERM = GIFT16_BITPERM


    rounds = []
    n_rounds = 5
    for r in range(n_rounds):
        S_in, S_out = model.add_round(sbox_size=SBOX_SIZE, n_sboxes=N_SBOXES)
        rounds.append((S_in, S_out))
        model.add_sbox_layer(S_in, S_out, sbox_constraints, sbox_size=SBOX_SIZE, n_boxes=N_SBOXES)
    for r in range(n_rounds - 1):
        model.connect_rounds_with_permutation(rounds[r][1], rounds[r + 1][0], BITPERM)

    for i in range(BLOCKSIZE):
        if i >= 1:
            model.solver.Add(rounds[0][0][i] == 1)
        else:
            model.solver.Add(rounds[0][0][i] == 0)


    model.set_objective([1] * BLOCKSIZE, rounds[n_rounds - 1][1])


    model.solve_for_integral_distinguisher(rounds[n_rounds - 1][1], blocksize=BLOCKSIZE)
    classic_16_solve_times.append(sum(model.solve_times))
print(classic_16_solve_times)

[i] Solver threads set to 1
[+] Solve #0 -> Objective: 1.0
  └─ Forcing x_4_out_9 = 0
[+] Solve #1 -> Objective: 2.0

[V] Integral Distinguisher FOUND!
Indices with value 1: [0, 8]
Zero-forced bits:
  x_4_out_9
[i] Solver threads set to 1
[+] Solve #0 -> Objective: 1.0
  └─ Forcing x_4_out_9 = 0
[+] Solve #1 -> Objective: 2.0

[V] Integral Distinguisher FOUND!
Indices with value 1: [0, 8]
Zero-forced bits:
  x_4_out_9
[i] Solver threads set to 1
[+] Solve #0 -> Objective: 1.0
  └─ Forcing x_4_out_9 = 0
[+] Solve #1 -> Objective: 2.0

[V] Integral Distinguisher FOUND!
Indices with value 1: [0, 8]
Zero-forced bits:
  x_4_out_9
[i] Solver threads set to 1
[+] Solve #0 -> Objective: 1.0
  └─ Forcing x_4_out_9 = 0
[+] Solve #1 -> Objective: 2.0

[V] Integral Distinguisher FOUND!
Indices with value 1: [0, 8]
Zero-forced bits:
  x_4_out_9
[i] Solver threads set to 1
[+] Solve #0 -> Objective: 1.0
  └─ Forcing x_4_out_9 = 0
[+] Solve #1 -> Objective: 2.0

[V] Integral Distinguisher FOUND!
Indi

# Application to GIFT-32

In [12]:
for idx in range(10):
    model = GenMILPModel("SCIP")

    SBOX_SIZE = 4
    N_SBOXES = 8
    BLOCKSIZE = SBOX_SIZE * N_SBOXES
    BITPERM = GIFT32_BITPERM


    rounds = []
    n_rounds = 8
    for r in range(n_rounds):
        S_in, S_out = model.add_round(sbox_size=SBOX_SIZE, n_sboxes=N_SBOXES)
        rounds.append((S_in, S_out))
        model.add_sbox_layer(S_in, S_out, sbox_constraints, sbox_size=SBOX_SIZE, n_boxes=N_SBOXES)
    for r in range(n_rounds - 1):
        model.connect_rounds_with_permutation(rounds[r][1], rounds[r + 1][0], BITPERM)

    for i in range(BLOCKSIZE):
        if i >= 1:
            model.solver.Add(rounds[0][0][i] == 1)
        else:
            model.solver.Add(rounds[0][0][i] == 0)


    model.set_objective([1] * BLOCKSIZE, rounds[n_rounds - 1][1])


    model.solve_for_integral_distinguisher(rounds[n_rounds - 1][1], blocksize=BLOCKSIZE)
    classic_32_solve_times.append(sum(model.solve_times))
print(classic_32_solve_times)

[i] Solver threads set to 1
[+] Solve #0 -> Objective: 1.0
  └─ Forcing x_7_out_16 = 0
[+] Solve #1 -> Objective: 1.0
  └─ Forcing x_7_out_25 = 0
[+] Solve #2 -> Objective: 1.0
  └─ Forcing x_7_out_13 = 0
[+] Solve #3 -> Objective: 1.0
  └─ Forcing x_7_out_20 = 0
[+] Solve #4 -> Objective: 1.0
  └─ Forcing x_7_out_17 = 0
[+] Solve #5 -> Objective: 1.0
  └─ Forcing x_7_out_12 = 0
[+] Solve #6 -> Objective: 1.0
  └─ Forcing x_7_out_21 = 0
[+] Solve #7 -> Objective: 1.0
  └─ Forcing x_7_out_24 = 0
[+] Solve #8 -> Objective: 1.0
  └─ Forcing x_7_out_5 = 0
[+] Solve #9 -> Objective: 1.0
  └─ Forcing x_7_out_28 = 0
[+] Solve #10 -> Objective: 1.0
  └─ Forcing x_7_out_1 = 0
[+] Solve #11 -> Objective: 1.0
  └─ Forcing x_7_out_29 = 0
[+] Solve #12 -> Objective: 0.9999999999999999
  └─ Forcing x_7_out_9 = 0
[+] Solve #13 -> Objective: 1.0
  └─ Forcing x_7_out_0 = 0
[+] Solve #14 -> Objective: 1.0
  └─ Forcing x_7_out_4 = 0
[+] Solve #15 -> Objective: 1.0
  └─ Forcing x_7_out_30 = 0
[+] Solve #1

[+] Solve #9 -> Objective: 1.0
  └─ Forcing x_7_out_28 = 0
[+] Solve #10 -> Objective: 1.0
  └─ Forcing x_7_out_1 = 0
[+] Solve #11 -> Objective: 1.0
  └─ Forcing x_7_out_29 = 0
[+] Solve #12 -> Objective: 0.9999999999999999
  └─ Forcing x_7_out_9 = 0
[+] Solve #13 -> Objective: 1.0
  └─ Forcing x_7_out_0 = 0
[+] Solve #14 -> Objective: 1.0
  └─ Forcing x_7_out_4 = 0
[+] Solve #15 -> Objective: 1.0
  └─ Forcing x_7_out_30 = 0
[+] Solve #16 -> Objective: 1.0
  └─ Forcing x_7_out_8 = 0
[+] Solve #17 -> Objective: 1.0
  └─ Forcing x_7_out_6 = 0
[+] Solve #18 -> Objective: 1.0
  └─ Forcing x_7_out_18 = 0
[+] Solve #19 -> Objective: 1.0000000000000004

[V] Integral Distinguisher FOUND!
Indices with value 1: [2]
Zero-forced bits:
  x_7_out_16
  x_7_out_25
  x_7_out_13
  x_7_out_20
  x_7_out_17
  x_7_out_12
  x_7_out_21
  x_7_out_24
  x_7_out_5
  x_7_out_28
  x_7_out_1
  x_7_out_29
  x_7_out_9
  x_7_out_0
  x_7_out_4
  x_7_out_30
  x_7_out_8
  x_7_out_6
  x_7_out_18
[i] Solver threads set to 

# Application to GIFT-64

In [5]:
for idx in range(10):
    model = GenMILPModel("SCIP")

    SBOX_SIZE = 4
    N_SBOXES = 16
    BLOCKSIZE = SBOX_SIZE * N_SBOXES
    BITPERM = GIFT64_BITPERM


    rounds = []
    n_rounds = 9
    for r in range(n_rounds):
        S_in, S_out = model.add_round(sbox_size=SBOX_SIZE, n_sboxes=N_SBOXES)
        rounds.append((S_in, S_out))
        model.add_sbox_layer(S_in, S_out, sbox_constraints, sbox_size=SBOX_SIZE, n_boxes=N_SBOXES)
    for r in range(n_rounds - 1):
        model.connect_rounds_with_permutation(rounds[r][1], rounds[r + 1][0], BITPERM)

    for i in range(BLOCKSIZE):
        if i >= 1:
            model.solver.Add(rounds[0][0][i] == 1)
        else:
            model.solver.Add(rounds[0][0][i] == 0)


    model.set_objective([1] * BLOCKSIZE, rounds[n_rounds - 1][1])


    model.solve_for_integral_distinguisher(rounds[n_rounds - 1][1], blocksize=BLOCKSIZE)
    classic_64_solve_times.append(sum(model.solve_times))
    print(sum(model.solve_times))
print(classic_64_solve_times)

[i] Solver threads set to 1
[+] Solve #0 -> Objective: 1.0
  └─ Forcing x_8_out_29 = 0
[+] Solve #1 -> Objective: 1.0
  └─ Forcing x_8_out_56 = 0
[+] Solve #2 -> Objective: 1.0
  └─ Forcing x_8_out_45 = 0
[+] Solve #3 -> Objective: 1.0
  └─ Forcing x_8_out_44 = 0
[+] Solve #4 -> Objective: 1.0
  └─ Forcing x_8_out_21 = 0
[+] Solve #5 -> Objective: 1.0
  └─ Forcing x_8_out_41 = 0
[+] Solve #6 -> Objective: 1.0
  └─ Forcing x_8_out_13 = 0
[+] Solve #7 -> Objective: 1.0
  └─ Forcing x_8_out_20 = 0
[+] Solve #8 -> Objective: 1.0
  └─ Forcing x_8_out_36 = 0
[+] Solve #9 -> Objective: 1.0
  └─ Forcing x_8_out_40 = 0
[+] Solve #10 -> Objective: 1.0
  └─ Forcing x_8_out_52 = 0
[+] Solve #11 -> Objective: 1.0
  └─ Forcing x_8_out_0 = 0
[+] Solve #12 -> Objective: 1.0
  └─ Forcing x_8_out_9 = 0
[+] Solve #13 -> Objective: 1.0
  └─ Forcing x_8_out_25 = 0
[+] Solve #14 -> Objective: 1.0
  └─ Forcing x_8_out_53 = 0
[+] Solve #15 -> Objective: 1.0
  └─ Forcing x_8_out_57 = 0
[+] Solve #16 -> Objecti

[+] Solve #6 -> Objective: 1.0
  └─ Forcing x_8_out_13 = 0
[+] Solve #7 -> Objective: 1.0
  └─ Forcing x_8_out_20 = 0
[+] Solve #8 -> Objective: 1.0
  └─ Forcing x_8_out_36 = 0
[+] Solve #9 -> Objective: 1.0
  └─ Forcing x_8_out_40 = 0
[+] Solve #10 -> Objective: 1.0
  └─ Forcing x_8_out_52 = 0
[+] Solve #11 -> Objective: 1.0
  └─ Forcing x_8_out_0 = 0
[+] Solve #12 -> Objective: 1.0
  └─ Forcing x_8_out_9 = 0
[+] Solve #13 -> Objective: 1.0
  └─ Forcing x_8_out_25 = 0
[+] Solve #14 -> Objective: 1.0
  └─ Forcing x_8_out_53 = 0
[+] Solve #15 -> Objective: 1.0
  └─ Forcing x_8_out_57 = 0
[+] Solve #16 -> Objective: 1.0
  └─ Forcing x_8_out_33 = 0
[+] Solve #17 -> Objective: 1.0
  └─ Forcing x_8_out_37 = 0
[+] Solve #18 -> Objective: 1.0
  └─ Forcing x_8_out_12 = 0
[+] Solve #19 -> Objective: 1.0
  └─ Forcing x_8_out_4 = 0
[+] Solve #20 -> Objective: 1.0
  └─ Forcing x_8_out_8 = 0
[+] Solve #21 -> Objective: 1.0
  └─ Forcing x_8_out_5 = 0
[+] Solve #22 -> Objective: 1.0
  └─ Forcing x_8_

[+] Solve #12 -> Objective: 1.0
  └─ Forcing x_8_out_9 = 0
[+] Solve #13 -> Objective: 1.0
  └─ Forcing x_8_out_25 = 0
[+] Solve #14 -> Objective: 1.0
  └─ Forcing x_8_out_53 = 0
[+] Solve #15 -> Objective: 1.0
  └─ Forcing x_8_out_57 = 0
[+] Solve #16 -> Objective: 1.0
  └─ Forcing x_8_out_33 = 0
[+] Solve #17 -> Objective: 1.0
  └─ Forcing x_8_out_37 = 0
[+] Solve #18 -> Objective: 1.0
  └─ Forcing x_8_out_12 = 0
[+] Solve #19 -> Objective: 1.0
  └─ Forcing x_8_out_4 = 0
[+] Solve #20 -> Objective: 1.0
  └─ Forcing x_8_out_8 = 0
[+] Solve #21 -> Objective: 1.0
  └─ Forcing x_8_out_5 = 0
[+] Solve #22 -> Objective: 1.0
  └─ Forcing x_8_out_49 = 0
[+] Solve #23 -> Objective: 1.0
  └─ Forcing x_8_out_28 = 0
[+] Solve #24 -> Objective: 1.0
  └─ Forcing x_8_out_48 = 0
[+] Solve #25 -> Objective: 1.0
  └─ Forcing x_8_out_60 = 0
[+] Solve #26 -> Objective: 1.0
  └─ Forcing x_8_out_24 = 0
[+] Solve #27 -> Objective: 1.0
  └─ Forcing x_8_out_32 = 0
[+] Solve #28 -> Objective: 1.0
  └─ Forcing

[+] Solve #23 -> Objective: 1.0
  └─ Forcing x_8_out_28 = 0
[+] Solve #24 -> Objective: 1.0
  └─ Forcing x_8_out_48 = 0
[+] Solve #25 -> Objective: 1.0
  └─ Forcing x_8_out_60 = 0
[+] Solve #26 -> Objective: 1.0
  └─ Forcing x_8_out_24 = 0
[+] Solve #27 -> Objective: 1.0
  └─ Forcing x_8_out_32 = 0
[+] Solve #28 -> Objective: 1.0
  └─ Forcing x_8_out_17 = 0
[+] Solve #29 -> Objective: 1.0
  └─ Forcing x_8_out_16 = 0
[+] Solve #30 -> Objective: 1.0
  └─ Forcing x_8_out_1 = 0
[+] Solve #31 -> Objective: 1.0
  └─ Forcing x_8_out_54 = 0
[+] Solve #32 -> Objective: 1.0
  └─ Forcing x_8_out_61 = 0
[+] Solve #33 -> Objective: 1.0
  └─ Forcing x_8_out_18 = 0
[+] Solve #34 -> Objective: 2.0

[V] Integral Distinguisher FOUND!
Indices with value 1: [14, 38]
Zero-forced bits:
  x_8_out_29
  x_8_out_56
  x_8_out_45
  x_8_out_44
  x_8_out_21
  x_8_out_41
  x_8_out_13
  x_8_out_20
  x_8_out_36
  x_8_out_40
  x_8_out_52
  x_8_out_0
  x_8_out_9
  x_8_out_25
  x_8_out_53
  x_8_out_57
  x_8_out_33
  x_8_

In [8]:
sum(model.solve_times)

59.377745389938354

# Quantum Experiments

In [11]:

import re
import time
from typing import List, Sequence, Tuple, Optional

import dimod
from dimod import ConstrainedQuadraticModel, Binary
from dwave.system import LeapHybridCQMSampler


class GenCQMModel:
    def __init__(self, solver_name: Optional[str] = "hybrid_constrained_quadratic_model_version1p",
                 label: Optional[str] = None):
        self.cqm = ConstrainedQuadraticModel()
        self.rounds: List[dict] = []  # Each: { 'in': [vars], 'out': [vars] }
        self.constraints = []         # Keep labels for reference
        self.solve_times: List[float] = []
        self.solver_name = solver_name
        self.label = label
        self.sampleset = None
        self.best_feasible = None  # dimod.SampleSet row-like (sampleset.first)
        self._objective_maximized = False  # track if we negated objective

    def _var_label(self, v) -> str:
        try:
            return next(iter(v.variables))
        except Exception:
            return getattr(v, 'name', str(v))

    def _var_value(self, sample: dict, v) -> int:
        return int(sample[self._var_label(v)])

    def add_round(self, sbox_size: int, n_sboxes: int, prefix: str = "x") -> Tuple[Sequence[Binary], Sequence[Binary]]:
        round_idx = len(self.rounds)
        n_bits = sbox_size * n_sboxes
        S_in = [Binary(f"{prefix}_{round_idx}_in_{i}") for i in range(n_bits)]
        S_out = [Binary(f"{prefix}_{round_idx}_out_{i}") for i in range(n_bits)]
        self.rounds.append({'in': S_in, 'out': S_out})
        return S_in, S_out

    def connect_rounds_with_permutation(self, prev_out: Sequence[Binary], next_in: Sequence[Binary], perm: Sequence[int]):
        assert len(prev_out) == len(next_in) == len(perm)
        for i, j in enumerate(perm):
            # Enforce next_in[j] == prev_out[i]
            lbl = f"perm_{len(self.constraints)}_{i}"
            self.cqm.add_constraint(next_in[j] - prev_out[i] == 0, label=lbl)
            self.constraints.append(lbl)

    def add_constraint(self, coeffs: Sequence[int], vars: Sequence[Binary], rhs: int, sense: str = '==', label: Optional[str] = None):
        lhs = sum(c * v for c, v in zip(coeffs, vars))
        if sense == '==':
            ct = self.cqm.add_constraint(lhs == rhs, label=label)
        elif sense == '<=':
            ct = self.cqm.add_constraint(lhs <= rhs, label=label)
        elif sense == '>=':
            ct = self.cqm.add_constraint(lhs >= rhs, label=label)
        else:
            raise ValueError(f"Unsupported constraint type: {sense}")
        self.constraints.append(ct)
        return ct

    def add_constraint_from_string(self, expr: str, a_vars: Sequence[Binary], b_vars: Sequence[Binary], label_prefix: Optional[str] = None):
        if '>=' in expr:
            lhs, rhs = expr.split(">=")
            sense = '>='
        elif '<=' in expr:
            lhs, rhs = expr.split("<=")
            sense = '<='
        elif '==' in expr:
            lhs, rhs = expr.split("==")
            sense = '=='
        else:
            raise ValueError("Constraint must contain >=, <=, or ==")

        rhs = int(rhs.strip())
        pattern = re.compile(r'([+-]?\s*\d*\s*\*\s*[ab]\[\d+\]|[+-]?\s*[ab]\[\d+\])')
        terms = pattern.findall(lhs)

        coeffs = []
        vars_ = []

        for term in terms:
            term = term.replace(" ", "")
            if '*' in term:
                coef_str, var_str = term.split('*')
                if coef_str in ('', '+'):
                    coef = 1
                elif coef_str == '-':
                    coef = -1
                else:
                    coef = int(coef_str)
            else:
                coef = -1 if term.startswith('-') else 1
                var_str = term.lstrip('+-')

            var_type = var_str[0]
            index = int(var_str[2:-1])

            if var_type == 'a':
                v = a_vars[index]
            elif var_type == 'b':
                v = b_vars[index]
            else:
                raise ValueError(f"Unknown variable type: {var_type}")

            coeffs.append(coef)
            vars_.append(v)

        lbl = f"ct_{label_prefix or 'sbox'}_{len(self.constraints)}"
        return self.add_constraint(coeffs, vars_, rhs, sense, label=lbl)

    def add_sbox_layer(self, input_vars: Sequence[Binary], output_vars: Sequence[Binary],
                       sbox_constraints: Sequence[str], sbox_size: int, n_boxes: int,
                       label_prefix: str = "sbox"):
        assert len(input_vars) == len(output_vars) == sbox_size * n_boxes
        for i in range(n_boxes):
            a_block = input_vars[i * sbox_size:(i + 1) * sbox_size]
            b_block = output_vars[i * sbox_size:(i + 1) * sbox_size]
            for expr in sbox_constraints:
                self.add_constraint_from_string(expr, a_block, b_block,
                                                label_prefix=f"{label_prefix}_{i}")

    # ------------------------------
    # Objective
    # ------------------------------
    def set_objective(self, coeffs: Sequence[int], vars: Sequence[Binary], sense: str = 'min'):
        expr = sum(c * v for c, v in zip(coeffs, vars))
        if sense == 'min':
            self.cqm.set_objective(expr)
            self._objective_maximized = False
        elif sense == 'max':
            # CQM is a minimizer; to maximize f, minimize -f
            self.cqm.set_objective(-expr)
            self._objective_maximized = True
        else:
            raise ValueError(f"Unsupported objective direction: {sense}")
            
    def set_objective_constraint(self, coeffs: Sequence[int], vars: Sequence[Binary], obj_bound=None):
        if obj_bound != None:
            expr = sum(c * v for c, v in zip(coeffs, vars))
            self.cqm.add_constraint(expr==obj_bound)

    # ------------------------------
    # Solve utilities
    # ------------------------------
    def _get_sampler(self) -> LeapHybridCQMSampler:
        if self.solver_name:
            return LeapHybridCQMSampler(solver=self.solver_name)
        return LeapHybridCQMSampler()

    def solve(self, time_limit_sec: Optional[int] = None, label: Optional[str] = None, time_limit=None):
        sampler = self._get_sampler()

        t0 = time.time()
        self.sampleset = sampler.sample_cqm(self.cqm, label=label or self.label, time_limit=time_limit)
        self.solve_times.append(time.time() - t0)

        # Prefer feasible if available
        try:
            feasible = self.sampleset.filter(lambda d: d.is_feasible)
        except Exception:
            feasible = None

        if feasible and len(feasible):
            self.best_feasible = feasible.first
        else:
            self.best_feasible = self.sampleset.first  # may be infeasible; still informative
        return self.best_feasible

    def get_solution(self) -> List[dict]:
        if self.best_feasible is None:
            raise RuntimeError("Call solve() first.")
        sample = self.best_feasible.sample
        sol = []
        for round_data in self.rounds:
            sol.append({
                key: [self._var_value(sample, v) for v in round_data[key]]
                for key in ('in', 'out')
            })
        return sol

    def print_solution(self):
        if self.best_feasible is None:
            raise RuntimeError("Call solve() first.")
        row = self.best_feasible
        obj = -row.energy if self._objective_maximized else row.energy
        print("Objective value:", obj)
        for r, round_data in enumerate(self.rounds):
            for key in ('in', 'out'):
                bits = [self._var_value(row.sample, v) for v in round_data[key]]
                print(f"Round {r} {key}: {bits}")

    # ------------------------------
    # Integral-distinguisher style loop (iterative zero-forcing)
    # ------------------------------
    def solve_for_integral_distinguisher(self, target_vars: Sequence[Binary], blocksize: int = 64,
                                         time_limit_sec: Optional[int] = None, verbose: bool = True, time_limit = None):
        """Mimics the MILP loop: repeatedly solve; if objective ≤ 1, force one 1‑bit to 0 and repeat.

        IMPORTANT: For this to make sense, set the objective beforehand as
            Minimize( sum(target_vars) )
        so that an objective value > 1 implies a multi‑bit active pattern.
        """
        counter = 0
        forced_labels = []
        found = Falsev 
        
        integral_bits = []
        self.solve_times.clear()

        while counter < blocksize:
            row = self.solve(time_limit=time_limit)
            energy = row.energy
            obj_val = -energy if self._objective_maximized else energy
            if verbose:
                print(f"[+] Solve #{counter} -> Objective: {obj_val}")

            # Get current assignment for target vars
            sample = row.sample
            ones_now = [i for i, v in enumerate(target_vars) if self._var_value(sample, v) == 1]

            # Heuristic stop rule (adapted from your MILP version)
            if obj_val > 1 or (len(ones_now) > 1):
                found = True
                integral_bits = ones_now
                break

            # Otherwise, force a 1‑bit to 0 and iterate
            progressed = False
            for v in target_vars:
                if self._var_value(sample, v) == 1:
                    lbl = f"force_zero_{self._var_label(v)}_{len(forced_labels)}"
                    self.cqm.add_constraint(v == 0, label=lbl)
                    forced_labels.append(lbl)
                    if verbose:
                        print(f"  └─ Forcing {self._var_label(v)} = 0")
                    counter += 1
                    progressed = True
                    break

            if not progressed:
                # Either already all zeros or infeasible; treat as success condition
                if verbose:
                    print("[X] No 1‑bits to force; treating as infeasible or trivial case.")
                found = True
                break

        if found:
            print("\n[V] Integral Distinguisher FOUND!")
            if integral_bits:
                print("Integral bits (indices with value 1):", integral_bits)
        else:
            print("\n[X] No integral distinguisher exists.")

        if verbose:
            print("Zero‑forced constraints:")
            if forced_labels:
                for name in forced_labels:
                    print(" ", name)
            elif counter == 0:
                print(" (none — distinguisher confirmed without forcing any bit)")
            else:
                print(" (none — model was immediately infeasible)")



# Application to GIFT-16

In [12]:
for idx in range(10):
    model = GenCQMModel(
        solver_name="hybrid_constrained_quadratic_model_version1p",
        label="gift16-cqm-test"
    )

    SBOX_SIZE = 4
    N_SBOXES = 4
    BLOCKSIZE = SBOX_SIZE * N_SBOXES
    BITPERM = GIFT16_BITPERM

    rounds = []
    n_rounds = 5

    for r in range(n_rounds):
        S_in, S_out = model.add_round(sbox_size=SBOX_SIZE, n_sboxes=N_SBOXES)
        rounds.append((S_in, S_out))
        model.add_sbox_layer(S_in, S_out, sbox_constraints,
                             sbox_size=SBOX_SIZE, n_boxes=N_SBOXES)

    for r in range(n_rounds - 1):
        model.connect_rounds_with_permutation(rounds[r][1], rounds[r + 1][0], BITPERM)

    for i in range(BLOCKSIZE):
        model.cqm.add_constraint(rounds[0][0][i] == (1 if i >= 1 else 0), label=f"fix_in_{i}")

    model.set_objective([1] * BLOCKSIZE, rounds[n_rounds - 1][1], sense='min')

    model.solve_for_integral_distinguisher(
        rounds[n_rounds - 1][1],
        blocksize=BLOCKSIZE,
        verbose=True
    )
    quantum_16_solve_times.append(sum(model.solve_times))
print(quantum_16_solve_times)

[+] Solve #0 -> Objective: 1.0
  └─ Forcing x_4_out_9 = 0
[+] Solve #1 -> Objective: 2.0

[V] Integral Distinguisher FOUND!
Integral bits (indices with value 1): [0, 13]
Zero‑forced constraints:
  force_zero_x_4_out_9_0
[+] Solve #0 -> Objective: 2.0

[V] Integral Distinguisher FOUND!
Integral bits (indices with value 1): [9, 12]
Zero‑forced constraints:
 (none — distinguisher confirmed without forcing any bit)
[+] Solve #0 -> Objective: 1.0
  └─ Forcing x_4_out_9 = 0
[+] Solve #1 -> Objective: 2.0

[V] Integral Distinguisher FOUND!
Integral bits (indices with value 1): [11, 12]
Zero‑forced constraints:
  force_zero_x_4_out_9_0
[+] Solve #0 -> Objective: 2.0

[V] Integral Distinguisher FOUND!
Integral bits (indices with value 1): [5, 8]
Zero‑forced constraints:
 (none — distinguisher confirmed without forcing any bit)
[+] Solve #0 -> Objective: 2.0

[V] Integral Distinguisher FOUND!
Integral bits (indices with value 1): [2, 4]
Zero‑forced constraints:
 (none — distinguisher confirmed w

In [13]:
sum(model.solve_times)

6.955843210220337

# Application to GIFT-32

In [14]:
for idx in range(10):
    model = GenCQMModel(
        solver_name="hybrid_constrained_quadratic_model_version1p",
        label="gift32-cqm-test"
    )

    SBOX_SIZE = 4
    N_SBOXES = 8
    BLOCKSIZE = SBOX_SIZE * N_SBOXES
    BITPERM = GIFT32_BITPERM

    rounds = []
    n_rounds = 8

    for r in range(n_rounds):
        S_in, S_out = model.add_round(sbox_size=SBOX_SIZE, n_sboxes=N_SBOXES)
        rounds.append((S_in, S_out))
        model.add_sbox_layer(S_in, S_out, sbox_constraints,
                             sbox_size=SBOX_SIZE, n_boxes=N_SBOXES)

    for r in range(n_rounds - 1):
        model.connect_rounds_with_permutation(rounds[r][1], rounds[r + 1][0], BITPERM)

    for i in range(BLOCKSIZE):
        model.cqm.add_constraint(rounds[0][0][i] == (1 if i >= 1 else 0), label=f"fix_in_{i}")

    model.set_objective([1] * BLOCKSIZE, rounds[n_rounds - 1][1], sense='min')

    model.solve_for_integral_distinguisher(
        rounds[n_rounds - 1][1],
        blocksize=BLOCKSIZE,
        verbose=True
    )
    quantum_32_solve_times.append(sum(model.solve_times))
print(quantum_32_solve_times)

[+] Solve #0 -> Objective: 2.0

[V] Integral Distinguisher FOUND!
Integral bits (indices with value 1): [1, 24]
Zero‑forced constraints:
 (none — distinguisher confirmed without forcing any bit)
[+] Solve #0 -> Objective: 2.0

[V] Integral Distinguisher FOUND!
Integral bits (indices with value 1): [0, 29]
Zero‑forced constraints:
 (none — distinguisher confirmed without forcing any bit)
[+] Solve #0 -> Objective: 3.0

[V] Integral Distinguisher FOUND!
Integral bits (indices with value 1): [13, 21, 30]
Zero‑forced constraints:
 (none — distinguisher confirmed without forcing any bit)
[+] Solve #0 -> Objective: 3.0

[V] Integral Distinguisher FOUND!
Integral bits (indices with value 1): [1, 15, 24]
Zero‑forced constraints:
 (none — distinguisher confirmed without forcing any bit)
[+] Solve #0 -> Objective: 2.0

[V] Integral Distinguisher FOUND!
Integral bits (indices with value 1): [5, 29]
Zero‑forced constraints:
 (none — distinguisher confirmed without forcing any bit)
[+] Solve #0 -> 

In [15]:
sum(model.solve_times)

4.72429347038269

# Application to GIFT-64

In [16]:
for idx in range(10):
    model = GenCQMModel(
        solver_name="hybrid_constrained_quadratic_model_version1p",
        label="gift64-cqm-test"
    )

    SBOX_SIZE = 4
    N_SBOXES = 16
    BLOCKSIZE = SBOX_SIZE * N_SBOXES
    BITPERM = GIFT64_BITPERM

    rounds = []
    n_rounds = 9

    for r in range(n_rounds):
        S_in, S_out = model.add_round(sbox_size=SBOX_SIZE, n_sboxes=N_SBOXES)
        rounds.append((S_in, S_out))
        model.add_sbox_layer(S_in, S_out, sbox_constraints,
                             sbox_size=SBOX_SIZE, n_boxes=N_SBOXES)

    for r in range(n_rounds - 1):
        model.connect_rounds_with_permutation(rounds[r][1], rounds[r + 1][0], BITPERM)

    for i in range(BLOCKSIZE):
        model.cqm.add_constraint(rounds[0][0][i] == (1 if i >= 1 else 0), label=f"fix_in_{i}")

    model.set_objective([1] * BLOCKSIZE, rounds[n_rounds - 1][1], sense='min')

    model.solve_for_integral_distinguisher(
        rounds[n_rounds - 1][1],
        blocksize=BLOCKSIZE,
        verbose=True
    )
    quantum_64_solve_times.append(sum(model.solve_times))
print(quantum_64_solve_times)

[+] Solve #0 -> Objective: 8.0

[V] Integral Distinguisher FOUND!
Integral bits (indices with value 1): [5, 11, 16, 21, 25, 32, 57, 60]
Zero‑forced constraints:
 (none — distinguisher confirmed without forcing any bit)
[+] Solve #0 -> Objective: 7.0

[V] Integral Distinguisher FOUND!
Integral bits (indices with value 1): [2, 10, 17, 37, 46, 49, 63]
Zero‑forced constraints:
 (none — distinguisher confirmed without forcing any bit)
[+] Solve #0 -> Objective: 7.0

[V] Integral Distinguisher FOUND!
Integral bits (indices with value 1): [2, 5, 14, 21, 34, 53, 62]
Zero‑forced constraints:
 (none — distinguisher confirmed without forcing any bit)
[+] Solve #0 -> Objective: 7.0

[V] Integral Distinguisher FOUND!
Integral bits (indices with value 1): [7, 13, 18, 29, 43, 49, 58]
Zero‑forced constraints:
 (none — distinguisher confirmed without forcing any bit)
[+] Solve #0 -> Objective: 6.0

[V] Integral Distinguisher FOUND!
Integral bits (indices with value 1): [8, 23, 29, 32, 40, 49]
Zero‑forc

In [17]:
sum(model.solve_times)

7.485617637634277